In [6]:
%run read_data.ipynb

In [7]:
script_dir = os.path.dirname('__file__')
results_dir = os.path.join(script_dir, 'Indv_train/')
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)
    
#script_dir = os.path.dirname('__file__')
results_dir1 = os.path.join(script_dir, 'Indv_train_CDF/')
if not os.path.isdir(results_dir1):
    os.makedirs(results_dir1)
    
#script_dir = os.path.dirname('__file__')
results_dir2 = os.path.join(script_dir, 'Indv_train_count/')
if not os.path.isdir(results_dir2):
    os.makedirs(results_dir2)
    
#script_dir = os.path.dirname('__file__')
results_dir3 = os.path.join(script_dir, 'Indv_train_box/')
if not os.path.isdir(results_dir3):
    os.makedirs(results_dir3)

In [8]:
%run functionv2.ipynb

In [ ]:
import timeit
start_time = timeit.default_timer()


import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

runtime_block = []
f = open("Train_data_length.txt", "a")
for i in range(8):
    runtime_block1 = []
    for j in range(3,61):
        data3 = rt_names[i].iloc[:,[0,j]]
        #print(data3)
        for l in range(54):
            if (i>3):
                data1 = data3[data3['TrainMaskNo']==wtt_d.iloc[l,0]]
            else:
                data1 = data3[data3['TrainMaskNo']==wtt_u.iloc[l,0]]
            data0 = data1.iloc[:,1]
            original_data1 = [k for k in data0 if str(k) != 'nan']
            if (len(original_data1)<25):
                continue
            original_data = outlier(original_data1)
            #print(original_data)
            if (len(original_data)<150):
                data2 = clt_conversion_10_per(data0)
            else:
                data2 = clt_conversion150_10(data0)
            data = outlier(data2)
            if (len(data)<25):
                print(str(j-2)+'.'+str(l+1)+' '+str(plot_type[i])+"Less 20",file=f)
                runtime_block1.append(np.nan)
                continue
            print(str(j-2)+'.'+str(l+1)+' '+str(plot_type[i])+"  Length of data=",len(data),file=f)
            t_dist = stats.t
            par_est = t_dist.fit(data)
            generated_data = t_dist.rvs(*par_est[:-2], loc=par_est[-2], scale=par_est[-1], size=len(data))
            #print(par_est[-2])
            #runtime_block1.append(math.floor(par_est[-2])+1)
            density_plot_train(data,original_data,generated_data,i,j,l)
            cdf_plot_train(data,original_data,generated_data,i,j,l)
            count_plot_train(data,original_data,generated_data,i,j,l)
            box_plot_train(data,original_data,generated_data,i,j,l)
    #runtime_block.append(runtime_block1)
#print(runtime_block)   
f.close()     
elapsed = timeit.default_timer() - start_time
print(elapsed)

In [ ]:
writer = pd.ExcelWriter('estimated_runtime.xlsx', engine='xlsxwriter')
types = ['_up_00','_up_01','_up_10','_up_11','_dn_00','_dn_01','_dn_10','_dn_11']
for i in range(8):
    pd.DataFrame({'Block':column1,'Run_time':runtime_block[i]}).to_excel(writer, sheet_name='runtime'+str(types[i]), index=False)
writer.save()